# **Choice-Learn: Linear Models Playground**

In [1]:
import numpy as np
import pandas as pd
from choice_learn.models import SimpleMNL, ConditionalLogit, NestedLogit
from choice_learn.data import ChoiceDataset
from choice_learn.datasets.base import load_heating, load_swissmetro

2025-05-24 05:34:30.934797: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## **Conditional Logit**

In [2]:
swiss_dataset = load_swissmetro(preprocessing="tutorial")

# Initialization of the model
swiss_model = ConditionalLogit(optimizer="lbfgs")

# Intercept for train & sm
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[0, 1])
# beta_he for train & sm
swiss_model.add_coefficients(feature_name="headway",
                             items_indexes=[0, 1],
                             coefficient_name="beta_he")
# beta_co for all items
swiss_model.add_coefficients(feature_name="cost",
                             items_indexes=[0, 1, 2])
# beta first_class for train
swiss_model.add_coefficients(feature_name="regular_class",
                             items_indexes=[0])
# beta seats for train
swiss_model.add_coefficients(feature_name="seats", items_indexes=[1])
# betas luggage for car
swiss_model.add_coefficients(feature_name="single_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage=1")
swiss_model.add_coefficients(feature_name="multiple_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage>1")
# beta TT only for car
swiss_model.add_coefficients(feature_name="travel_time",
                             items_indexes=[2],
                             coefficient_name="beta_tt_car")

# betas TT and HE shared by train and sm
swiss_model.add_shared_coefficient(feature_name="travel_time",
                                   items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="train_survey",
                                   items_indexes=[0, 1],
                                   coefficient_name="beta_survey")

# Estimation of the model
history = swiss_model.fit(swiss_dataset, get_report=True)

len(swiss_dataset) * swiss_model.evaluate(swiss_dataset)

Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function


<tf.Tensor: shape=(), dtype=float32, numpy=5159.3047>

In [4]:
swiss_model.report

,Coefficient Name,Coefficient Estimation,Std. Err,z_value,P(.>z)
0,beta_intercept_0,-1.292930,0.145000,-8.916751,0.000000e+00
1,beta_intercept_1,-0.502574,0.109712,-4.580841,4.631108e-06
2,beta_he_0,-0.314336,0.064734,-4.855809,1.198961e-06
3,beta_he_1,-0.377318,0.191370,-1.971670,4.864729e-02
4,beta_cost_0,-0.561762,0.094500,-5.944566,2.771898e-09
5,beta_cost_1,-0.281676,0.042252,-6.666521,2.619394e-11
6,beta_cost_2,-0.513847,0.101592,-5.057929,4.238341e-07
7,beta_regular_class,0.565017,0.079958,7.066464,1.589395e-12
8,beta_seats,-0.782448,0.085864,-9.112634,0.000000e+00
9,beta_luggage=1,0.422760,0.063327,6.675848,2.458078e-11


## **Multinomial Logit**

In [11]:
heating_df = load_heating(as_frame=True)

shared_features_by_choice = ["income", "agehed", "rooms"]
choice = ["depvar"]
items_features_by_choice = ["ic.", "oc."]
items = ["hp", "gc", "gr", "ec", "er"]

choices = np.array([items.index(val) for val in heating_df[choice].to_numpy().ravel()])
shared_features_by_choice = heating_df[shared_features_by_choice].to_numpy().astype("float32")
items_features_by_choice = np.stack([heating_df[[feat + item for feat in items_features_by_choice]].to_numpy() for item in items], axis=1)

dataset = ChoiceDataset(items_features_by_choice=items_features_by_choice,
                        choices=choices)
model = SimpleMNL(intercept=None)
history = model.fit(dataset, get_report=True, verbose=0)

print("Estimation Negative LogLikelihood:",
      model.evaluate(dataset) * len(dataset))

                                fit models needing them such as Conditional Logit.


Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function
Estimation Negative LogLikelihood: tf.Tensor(1095.2418, shape=(), dtype=float32)


In [5]:
model.report

,Coefficient Name,Coefficient Estimation,Std. Err,z_value,P(.>z)
0,Weights_items_features_0,-0.006232,0.000353,-17.665276,0.0
1,Weights_items_features_1,-0.004580,0.000322,-14.216596,0.0


## **Nested Logit**